In [1]:
import sys
sys.path.append('../../toyaikit/')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from openai import OpenAI

from toyaikit.chat import ChatAssistant, OpenAIClient, IPythonChatInterface
from toyaikit.tools import Tools

In [4]:
import os
import shutil

def start():
    project_name = input("Enter the new Django project name: ").strip()
    if not project_name:
        print("Project name cannot be empty.")
        return

    if os.path.exists(project_name):
        print(f"Directory '{project_name}' already exists. Please choose a different name or remove the existing directory.")
        return

    shutil.copytree('django_template', project_name)
    print(f"Django template copied to '{project_name}' directory.")

    return project_name

In [5]:
DEVELOPER_PROMPT = """
You are a coding agent. Your task is to modify the provided Django project template
according to user instructions. You don't tell the user what to do; you do it yourself using the 
available tools. First, think about the sequence of steps you will do, and then 
execute the sequence.
Always ensure changes are consistent with Django best practices and the project’s structure.

## Project Overview

The project is a Django 5.2.4 web application scaffolded with standard best practices. It uses:
- Python 3.8+
- Django 5.2.4 (as specified in pyproject.toml)
- uv for Python environment and dependency management
- SQLite as the default database (see settings.py)
- Standard Django apps and a custom app called myapp
- HTML templates for rendering views
- TailwindCSS for styling

## File Tree


├── .python-version
├── README.md
├── manage.py
├── pyproject.toml
├── uv.lock
├── myapp/
│   ├── __init__.py
│   ├── admin.py
│   ├── apps.py
│   ├── migrations/
│   │   └── __init__.py
│   ├── models.py
│   ├── templates/
│   │   └── home.html
│   ├── tests.py
│   └── views.py
├── myproject/
│   ├── __init__.py
│   ├── asgi.py
│   ├── settings.py
│   ├── urls.py
│   └── wsgi.py
└── templates/
    └── base.html

## Content Description

- manage.py: Standard Django management script for running commands.
- README.md: Setup and run instructions, including use of uv for dependency management.
- pyproject.toml: Project metadata and dependencies (Django 5.2.4).
- uv.lock: Lock file for reproducible Python environments.
- .python-version: Specifies the Python version for the project.
- myapp/: Custom Django app with models, views, admin, tests, and a template (home.html).
  - migrations/: Contains migration files for database schema.
- myproject/: Django project configuration (settings, URLs, WSGI/ASGI entrypoints).
  - settings.py: Configures installed apps, middleware, database (SQLite), templates, etc.
- templates/: Project-level templates, including base.html.

You have full access to modify, add, or remove files and code within this structure using your available tools.


## Additional instructions

- Don't execute "runproject", but you can execute other commands to check if the project is working.
- Make sure you use Tailwind styles for making the result look beatiful
- Use pictograms and emojis when possible. Font-awesome is awailable
- Avoid putting complex logic to templates - do it on the server side when possible
"""

In [6]:
project_name = start()

Enter the new Django project name:  todo-2


Django template copied to 'todo-2' directory.


In [14]:
project_name = 'todo'

In [7]:
from pathlib import Path
import tools

project_path = Path(project_name)
agent_tools = tools.AgentTools(project_path)

In [8]:
tools_obj = Tools()
tools_obj.add_tools(agent_tools)

In [9]:
tools_obj.get_tools()

[{'type': 'function',
  'name': 'execute_bash_command',
  'description': 'Execute a bash command in the shell and return its output, error, and exit code. Blocks running the Django development server (runserver).\n\nParameters:\n    command (str): The bash command to execute.\n    cwd (str, optional): Working directory to run the command in, relative to the project directory. Defaults to None.\nReturns:\n    tuple: (stdout (str), stderr (str), returncode (int))',
  'parameters': {'type': 'object',
   'properties': {'command': {'type': 'string',
     'description': 'command parameter'},
    'cwd': {'type': 'string', 'description': 'cwd parameter'}},
   'required': ['command'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'read_file',
  'description': 'Read and return the contents of a file at the given relative filepath.\n\nParameters:\n    filepath (str): Path to the file, relative to the project directory.\nReturns:\n    str: Contents of the file.',
  'parameter

In [10]:
chat_interface = IPythonChatInterface()
openai_client = OpenAIClient(client=OpenAI())

chat_assistant = ChatAssistant(
    tools=tools_obj,
    developer_prompt=DEVELOPER_PROMPT,
    chat_interface=chat_interface,
    llm_client=openai_client
)

In [11]:
chat_assistant.run()

You: todo app


You: stop


Chat ended.
